In [7]:
import geopandas as gpd
import pandas as pd
import requests
from shapely.geometry import Point
import numpy as np
from geopy.distance import geodesic

In [2]:
census_tracts = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp")
ga_tracts = census_tracts.to_crs(epsg=4326)


In [3]:
def find_census_tracts(lat, lon, census_tracts_gdf):
    """
    Identify the census tract containing the point and census tracts within 0.25 miles.
    """
    point = Point(lon, lat)
    
    # Ensure the census tracts are in WGS 84 (EPSG:4326) for point lookup
    census_tracts_gdf = census_tracts_gdf.to_crs(epsg=4326)
    actual_tract = census_tracts_gdf[census_tracts_gdf.contains(point)]
    
    # Convert to a projected CRS (meters) for distance calculations
    census_tracts_meters = census_tracts_gdf.to_crs(epsg=3857)
    point_meters = gpd.GeoSeries([point], crs=4326).to_crs(epsg=3857).iloc[0]
    
    # Buffer 0.25 miles (~402 meters) and find nearby census tracts
    point_buffer = point_meters.buffer(402)  # Distance in meters
    nearby_tracts = census_tracts_meters[census_tracts_meters.intersects(point_buffer)]
    
    tract_dict = {}
    if not actual_tract.empty:
        tract_dict["actual"] = actual_tract.iloc[0]["GEOID"]  # Assuming GEOID as census tract identifier
    
    for idx, row in nearby_tracts.iterrows():
        if row["GEOID"] != tract_dict.get("actual"):
            tract_dict[f"tract{idx}"] = row["GEOID"]
    
    return tract_dict

In [4]:
def calculate_indicators_score(tract_dict, indicators_df):
    """
    Calculate the number of indicators above the 50th percentile and determine the score.
    """
    indicators = [
        "above_median_Environmental Health Index",
        "above_median_Transit Access Index",
        "above_median_Percent of Population Above the Poverty Level",
        "above_median_Median Income",
        "above_median_Jobs Proximity Index"
    ]
    
    actual_tract = tract_dict.get("actual")
    
    # Ensure census tract column is treated as string for proper matching
    indicators_df["2020 Census Tract"] = indicators_df["2020 Census Tract"].astype(str)
    
    if actual_tract and actual_tract in indicators_df["2020 Census Tract"].values:
        actual_data = indicators_df[indicators_df["2020 Census Tract"] == actual_tract]
        actual_count = actual_data[indicators].sum(axis=1).iloc[0]  # Use .iloc[0] to extract value properly
    else:
        actual_count = 0
    
    near_counts = []
    for key, tract in tract_dict.items():
        if key == "actual":
            continue
        if tract in indicators_df["2020 Census Tract"].values:
            near_data = indicators_df[indicators_df["2020 Census Tract"] == tract]
            near_counts.append(near_data[indicators].sum(axis=1).iloc[0])
    
    near_max = max(near_counts) if near_counts else 0
    
    # Determine points based on the rules
    if actual_count >= 4:
        score = 10
    elif actual_count == 3:
        score = 8
    elif actual_count == 2:
        score = 6
    elif near_max >= 4:
        score = 9
    elif near_max == 3:
        score = 7
    elif near_max == 2:
        score = 5
    else:
        score = 0
    
    return {
        "actual_tract": actual_tract,
        "actual_count": actual_count,
        "nearby_max": near_max,
        "score": score
    }

# Load the census tracts shapefile
gdf = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp")
gdf = gdf.to_crs(epsg=4326)  

# Load the indicators dataset
indicators_df = pd.read_csv("../../data/processed/scoring_indicators/stable_communities_2024_processed.csv")

In [17]:
# Hertitage Competitive Core
latitude = 33.278968
longitude = -83.965148
tract_info = find_census_tracts(latitude, longitude, gdf)
score_info = calculate_indicators_score(tract_info, indicators_df)

print("Tract Information:", tract_info)
print("Score Information:", score_info)

Tract Information: {'actual': '13035150201'}
Score Information: {'actual_tract': '13035150201', 'actual_count': 1, 'nearby_max': 0, 'score': 0}


In [16]:
latitude = 32.579719
longitude = -83.700408
tract_info = find_census_tracts(latitude, longitude, gdf)
score_info = calculate_indicators_score(tract_info, indicators_df)

print("Tract Information:", tract_info)
print("Score Information:", score_info)

Tract Information: {'actual': '13153021123', 'tract824': '13153021122'}
Score Information: {'actual_tract': '13153021123', 'actual_count': 4, 'nearby_max': 4, 'score': 10}


In [14]:
# Example usage:
latitude = 32.45719
longitude = -81.796489
tract_info = find_census_tracts(latitude, longitude, gdf)
score_info = calculate_indicators_score(tract_info, indicators_df)

print("Tract Information:", tract_info)
print("Score Information:", score_info)

Tract Information: {'actual': '13031110202', 'tract851': '13031110301'}
Score Information: {'actual_tract': '13031110202', 'actual_count': 3, 'nearby_max': 3, 'score': 8}
